In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import *
import time
import RPi.GPIO as GPIO
import spidev


# setting up SPI for ADC
spi_ch = 0

# Enable SPI
spi = spidev.SpiDev(0, spi_ch)
spi.max_speed_hz = 1200000

def read_adc(adc_ch, vref = 3.3):

    # Make sure ADC channel is 0 or 1
    if adc_ch != 0:
        adc_ch = 1

    # Construct SPI message
    #  First bit (Start): Logic high (1)
    #  Second bit (SGL/DIFF): 1 to select single mode
    #  Third bit (ODD/SIGN): Select channel (0 or 1)
    #  Fourth bit (MSFB): 0 for LSB first
    #  Next 12 bits: 0 (don't care)
    msg = 0b11
    msg = ((msg << 1) + adc_ch) << 5
    msg = [msg, 0b00000000]
    reply = spi.xfer2(msg)

    # Construct single integer out of the reply (2 bytes)
    adc = 0
    for n in reply:
        adc = (adc << 8) + n

    # Last bit (0) is not part of ADC value, shift to remove it
    adc = adc >> 1

    return adc

in_pin = 15     # if set to HIGH, this pin goes in    
out_pin = 14    # if set to HIGH, this pin goes out

GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(in_pin, GPIO.OUT, initial=GPIO.LOW)#Motor Driver
GPIO.setup(out_pin, GPIO.OUT, initial=GPIO.LOW)#Motor Driver

def actuator_in():                      # retracts actuator, increases ADC
    GPIO.output(in_pin, GPIO.HIGH)
    GPIO.output(out_pin, GPIO.LOW)

def actuator_out():                     # extends actuator, decreases ADC
    GPIO.output(in_pin, GPIO.HIGH)
    GPIO.output(out_pin, GPIO.LOW)

filename = 'adc_by_time.csv'
adc_by_time = pd.read_csv(filename)
adc_by_time = adc_by_time.drop(columns='Unnamed: 0')


today_midnight = datetime.combine(date.today(), datetime.min.time())
# now = datetime.strptime(str(datetime.now()),"%Y-%m-%d %H:%M:%S.%f")
month = int(today_midnight.strftime("%m"))
time_striffed = today_midnight.strftime("%H:%M:%S")
time_stripped = datetime.strptime(time_striffed,"%H:%M:%S")

adj = timedelta(days=0,hours=0,minutes=10,seconds=0,microseconds=0)

month_df = adc_by_time.loc[adc_by_time['Month'] == month]

difference = 5   #time in seconds; will be 10*60*60

check = True
n=0
while check:
    new_df = month_df.loc[month_df['Time'] == time_striffed]     # find df to find time row
    if n > 0:
        last_desired = desired_adc
    desired_adc = int(new_df.to_numpy()[0][3])                   # find desired adc based on time
    current_adc = read_adc(0)                                    # find current adc value from channel 0
    print(current_adc)                                           # print it to check
    
    if last_desired != desired_adc:                                  # checks that panels have to move at all
        if desired_adc >= current_adc:                               # check if desired adc is greater than current
            check1 = True
            while check1:                                            # while check = true
                actuator_in()                                        # turn on actuator to retract
                if desired_adc <= current_adc:                       # when it reaches the desired value
                    check1 = False                                   # check = false, stop retracting

        elif desired_adc < current_adc:
            check2 = True
            while check1:
                actuator_out()
                if desired_adc >= current_adc:
                    check2 = False
    
    time_striffed = time_stripped.strftime("%H:%M:%S")           # convert to strif time
    time_stripped += adj                                         # add 10 minutes to time
    time.sleep(difference)
    n+=1
    if n>143:
        check = False                                            # stop running after 24 hrs


# while True:
#     adc_0 = read_adc(0)
# #     adc_1 = read_adc(1)      # not needed right now
#     actuator_out()
# #         actuator_in()
#     print(adc_0)
#     #time.sleep(2)

finally:
    spi.close()
#    GPIO.cleanup()

In [2]:
'''
filename = 'adc_by_time.csv'
adc_by_time = pd.read_csv(filename)

adc_by_time = adc_by_time.drop(columns='Unnamed: 0')
# adc_by_time.loc[500:550]
'''

In [28]:
import pandas as pd
import numpy as np
import datetime
from datetime import *
import time

filename = 'adc_by_time.csv'
adc_by_time = pd.read_csv(filename)
adc_by_time = adc_by_time.drop(columns='Unnamed: 0')

today_midnight = datetime.combine(date.today(), datetime.min.time())
# now = datetime.strptime(str(datetime.now()),"%Y-%m-%d %H:%M:%S.%f")
month = int(today_midnight.strftime("%m"))
time_striffed = today_midnight.strftime("%H:%M:%S")
time_stripped = datetime.strptime(time_striffed,"%H:%M:%S")

adj = timedelta(days=0,hours=0,minutes=10,seconds=0,microseconds=0)

# print(adc_by_time.loc[adc_by_time['Time'] == time_striffed])

month_df = adc_by_time.loc[adc_by_time['Month'] == month]
# month_df

difference = 2   #time in seconds; will be 10*60*60

check = True
n=0
while check:
    new_df = month_df.loc[month_df['Time'] == time_striffed]
    desired_adc = int(new_df.to_numpy()[0][3])
    time_striffed = time_stripped.strftime("%H:%M:%S")
    time_stripped += adj
    n+=1
    if n>143:
        check = False

429
429
429
429
429
429
429
429
429
429
429
429
429
429
429
429
429
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
688
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
892
877
862
847
831
815
799
782
765
747
729
711
693
674
655
636
616
597
577
557
537
517
496
476
456
436
429
429
429
429
429
429
429
429
429


In [7]:
# # import machine
 
# # analog_value = machine.ADC(28)    # pin 28 for ADC

# def adc_feedback():                 # checks if value reaches 536
#     check = True
    
#     while check:
#         reading = analog_value.read_u16()     
#         print("ADC: ",reading)
#         time.sleep(0.05)         # reads and prints ADC value every half second
#         return(False)
    
#         if reading >= 536:
#             check = False
#             return(True)
        

ModuleNotFoundError: No module named 'machine'

In [5]:
# pin = 14                          # using pin 11 for motor

# def stop():
#     gpio.setmode(gpio.BOARD)
#     gpio.setup(pin,gpio.OUT)
#     gpio.setwarnings(False)

#     gpio.output(pin, gpio.LOW)

# def actuator_on():
#     gpio.setmode(gpio.BOARD)
#     gpio.setup(pin,gpio.OUT)   
#     gpio.setwarnings(False)
#     gpio.output(pin, gpio.HIGH)   
    
#     check = True
#     while check:
#         hold = adc_feedback()    # checks adc_feedback until it reaches 536
#         if hold:
#             stop()
#             check = False